# Об одном методе решения задач оптимизации в ритейле

Привет, Habr! На связи отдел аналитики данных X5 Tech.

Сегодня мы поговорим об очень интересном разделе прикладной математики — оптимизации.
Так как данная тема достаточно обширна, то помимо данной статьи будет ещё 2:

    1) Сравнение open-source солверов и их применение в ритейле [статья](http://)
    2) Решение модельной задачи ценообразования оптимизаторами [статья](http://)


Цель данной статьи — рассказать про задачи, которые решаются методами оптимизации
и продемонстрировать применение пакета [__Pyomo__](http://www.pyomo.org/) для решения модельной задачи.


## Примеры задач


### Примеры задач из жизни каждого


Практически каждый человек ежедневно решает оптимизационные задачи даже не задумываясь об этом.

Например, мы, как правило, хотим минимизировать наши расходы для приобретения необходимых товаров, но при условии, чтобы эти товары были максимально полезны.
Полезность здесь у каждого своя:
для одних она определяется количеством растительных жиров,
для других — минимальной суммарной стоимостью корзины,
для других — наличием привычных товаров, и тд.

Другой пример — мы хотим распределить свой маршрут в отпуске так,
чтобы посетить всё, что запланировали с минимальными временными затратами на дорогу, и при этом не забыть позагорать на пляже.

Бизнес требования часто сводятся к задаче многокритериальной оптимизации и задаче оптимального управления.
В таких постановках решения, найденные методами оптимизации, чрезвычайно полезны для принятия решений.
Приведём для иллюстрации несколько задач из области ритейла.


### Примеры задач из жизни ритейла


**Оптимальное распределение маркетингового бюджета**

Реализовать максимально эффективно выделенный на маркетинговые активности бюджет.
Есть несколько каналов для рекламных акций, выделенный бюджет, цель — максимально выгодно инвестировать бюджет,
чтобы суммарный доход со всех коммуникаций был максимален.
Также необходимо учесть бизнес ограничения на нагрузку каждого канала + частоту взаимодействия.

**Ценообразование**

Поиск оптимальной конфигурации цен с учетом ценового позиционирования, допустимых ценовых диапазонов для каждого товара и набора бизнес-ограничений. Цены должны максимизировать суммарный доход, а прибыль быть не ниже на заданного уровня.

**Планирование ассортимента**

Подобрать уровни ассортимента мерчгрупп так, чтобы максимизировать оборот с учетом полочного пространства, выделенного на мерчгруппу.

**Закупка товаров**

Задача — распределить бюджет, выделенный на закупки, для поддержания товарооборота, достаточного уровня сервиса,
и при этом достигнуть определенных финансовых показателей под выделенный бюджет на закупки.

**Логистика**

Задача — найти оптимальный график доставки продуктов с учётом вместимости грузовиков, затрат на логистику и тд.


## Общая постановка задачи и её разновидности


Прежде всего рассмотрим постановку задачи в общем виде:

$x$ - вектор размерности $n$, $x \in X$ - допустимое множество значений этих переменных.

$f(x) \to \min(\max)$, $f(\cdot)$ - целевая функция

$g_i(x) \leqslant 0, \ i=1..m$ - ограничения вида неравенств

$h_i(x) = 0, \ j=1..k$ - ограничения вида равенств

Исходя из практики можно разложить данную постановку на несколько классов в зависимости от вида целевой функции, ограничений и $X$:

* __Безусловная оптимизация__ $g_i(x), h_j(x)$ - отсутствуют, $X = \mathbb{R}^n$;

* __LP__ (linear programming) - линейное программирование. $f(x), g_i(x), h_j(x)$ - линейные функции, $X = \mathbb{R}_+^n$;

* __MILP__ (mixed integer linear programming) - смешанное целочисленное линейное программирование, это задача LP в которой только часть переменных являются целочисленными;

* __NLP__ (nonlinear programming) - нелинейное программирование, возникает когда хотя бы одна из функций $f(x),\ g_i(x),\ h_j(x)$ нелинейна;

* __MINLP__ (mixed integer nonlinear programming) - смешанное целочисленное нелинейное программирование, возникает как и в MILP, когда часть переменных принимает целочисленные значения;

__NLP__ в свою очередь можно подробить еще на множество разных классов в зависимости от вида нелинейности и выпуклости.


## Модельная задача оптимизации


### Пример модели с широкой и длинной матрицей

### Пример сгенерированный данных


<details>
<summary> |> Пример данных для MILP постановки </summary>
<div><pre><code class="python">
Пример данных для MILP постановки
# plu_line - код продуктовой линейки
# Ps - сетка цен для поиска
# Qs - сетка продаж для кажой цены из Ps
# xs - сетка индексов
# grid_size - размер сетки
# P_idx - индекс текущей цены в сетке. Если значение -1, то текщая цена не попала в сетку
data['nlp']

<img src="./images/data_milp_sample.png" width="1000" align="c"/>
</code></pre><p></p></div>
</details>


### Расчёт


На шаге 2 будет решаться сама оптимизационная задача.
Выглядит удобным описать оптимизационную модель в виде базового класса __OptimizationModel__,
который будет содержать методы, необходимые для постановки задачи.
Методы в классе __OptimizationModel__:
* init_objective - задание целевой функции,
* add_con_mrg - ограничений и метод,
* init_variables - задание переменных и границ.
* solve - поиск оптимального решения

Таким образом, можно организовать единый интерфейс для решения различными пакетами.
Реализация методов для каждой библиотеки будет отличаться, с чем можно ознакомиться в модуле optimizers.

Запуск оптимизационной задачи теперь можно описать в виде функции, в которую поступают данные,
модель для решения задачи, параметры ограничений — словарь и дополнительные опции для солвера [ссылка на github](https://github.com/mbudylin/OptimizersArticle/blob/main/optimizers/optimization.py).

```python
def pricing_optimization(data, opt_model, opt_params, solver, solver_option={}):
    """
    Запуск расчета оптимальных цен с помощью указанного класса оптимизатора и параметров
    :param data: входные данные для оптимизации
    :param opt_model: класс модели оптимизатора
    :param opt_params: параметры оптимизации
    :param solver: солвер для оптимизации
    :param solver_option: параметры солвера
    :return: словарь, возвращаемый моделью оптимизации
    """

    model = opt_model(data, opt_params['alpha'])

    model.init_variables()
    model.init_objective()
    # ...
    result = model.solve(solver=solver, options=solver_option)
    result['model_class'] = model
    return result
```

__Реализации классов для оптимизационных задач__

Описание классов здесь займет достаточно много места, детали можно найти [тут, код-github](https://github.com/mbudylin/OptimizersArticle/blob/main/optimizers/optimizers.py),
где реализовано 4 класса:
* __ScipyNlpOptimizationModel__ для NLP постановки через Scipy
* __PyomoNlpOptimizationModel__ для NLP постановки через Pyomo
* __PyomoLpOptimizationModel__ для MILP постановки через Pyomo
* __CvxpyLpOptimizationModel__ для MILP постановки через Cvxpy


## Как пользоваться


Чтобы запустить расчёты, нужно выполнить команду:

```console
python runner.py
```

Результаты замеров представлены ниже на графиках.


## Заключение

В данной статье мы рассмотрели, какие задачи оптимизации бывают на практике и научились решить
 их с помощью пакета __PYOMO__.

В следующей стать мы поговорим более детально про другие пакеты оптимизации для python и их различия.

Над статьёй работали Михаил Будылин, Антон Денисов, Михаил Дубровский. 

